In [ ]:
# default_exp __init__

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# Trading Specifications Fetcher

In [ ]:
from ccstabilizer import Binance
from ccstabilizer import MXC
from ccstabilizer import secrets

In [ ]:
# export
import os
import time
import yaml


class Fetcher(object):

    LIQUIDITY_YAML = './crypto-liquidity.yaml'
    TRADING_SPEC_YAML = './crypto-trading-spec.yaml'

    def __init__(self):
        self.binance = Binance(os.environ['BINANCE_API_KEY'], os.environ['BINANCE_API_SECRET'])
        self.mxc = MXC(os.environ['MXC_ACCESS_KEY'], os.environ['MXC_SECRET_KEY'])

    @staticmethod
    def write(yaml_data, yaml_file_path):
        with open(yaml_file_path, 'w') as f:
            yaml.dump(yaml_data, f, sort_keys=False)

    @staticmethod
    def read(yaml_file_path):
        with open(yaml_file_path) as f:
            return yaml.load(f, Loader=yaml.FullLoader)

    @staticmethod
    def extract_symbols(liquidity):
        symbol_and_liquid = liquidity.items()
        symbols = {
            symbol for symbol, _ in symbol_and_liquid
        }
        liquid_symbols = {
            symbol for symbol, liquid in symbol_and_liquid if liquid == True
        }
        return symbols, liquid_symbols

    def fetch_symbols(self):
        liquidity = {
            trading_spec['symbol']: trading_spec.get('limited', False) for trading_spec in self.mxc.get_symbols()
        }
        type(self).write(liquidity, type(self).LIQUIDITY_YAML)
        return type(self).extract_symbols(liquidity)

    def read_symbols(self):
        liquidity = type(self).read(type(self).LIQUIDITY_YAML)
        return type(self).extract_symbols(liquidity)

    def get_trading_spec(self, symbol):
        for trading_spec in self.mxc.get_symbols():
            if trading_spec.get('symbol', '') == symbol:
                break
        else:
            trading_spec = {}
        type(self).write(trading_spec, type(self).TRADING_SPEC_YAML)
        return trading_spec

In [ ]:
Fetcher().get_trading_spec('AR_USDT')

{'symbol': 'AR_USDT',
 'state': 'ENABLED',
 'price_scale': 3,
 'quantity_scale': 2,
 'min_amount': '5',
 'max_amount': '3000000',
 'maker_fee_rate': '0.002',
 'taker_fee_rate': '0.002',
 'limited': True,
 'etf_mark': 0}